In [1]:
%load_ext cython

In [2]:
import numpy as np

In [3]:
%%cython
# cython: infer_types = True
# cython: boundscheck = False
# cython: wraparound = False
from numpy import fft, sum as nsum, less_equal, zeros, conjugate, argmax, real, array
cimport cython
from libc.math cimport log, pow, exp, floor, ceil


cdef void mean_1d(const double[:] x, double* mean):
    cdef Py_ssize_t n = x.size, i

    for i in range(n):
        mean[0] += x[i]
    mean[0] /= n
    
    
cdef double gmean(const double[:] x):
    cdef Py_ssize_t n = x.size, i
    cdef double logsum = 0.0, prod = 1.0
    cdef double large = 1.e64, small = 1.e-64

    for i in range(n):
        prod *= x[i]
        if (prod > large) or (prod < small):
            logsum += log(prod)
            prod = 1.
    
    return exp((logsum + log(prod)) / n)


cdef linspace(double start, double stop, int N):
    cdef double[:] arr = zeros(N)
    cdef double step = (stop - start) / N
    cdef Py_ssize_t i

    for i in range(N):
        arr[i] = i * step + start
    
    return arr


cdef class FrequencyFeatures:
    # private attributes
    cdef bint base_run

    cdef Py_ssize_t M, N, P, i, j, k

    cdef double invlog2
    cdef double mean

    cdef double[:, :] maxf, maxfv, df_ratio, spec_flat, spec_ent

    cdef int nfft, ihcut, ilcut
    cdef long[:, :] imax
    cdef double[:] freq
    cdef double lic2, logps

    cdef complex[:, :, :] sp_hat
    cdef double[:, :, :] sp_norm
    
    def __init__(self):
        self.base_run = False
        self.invlog2 = 1 / log(2.0)

    cpdef _base_fn(self, const double[:, :, :] x, double fs, double low_cut, double hi_cut):
        # Book-keeping
        self.M = x.shape[0]
        self.N = x.shape[1]
        self.P = x.shape[2]

        self.maxf = zeros((self.M, self.P))
        self.maxfv = zeros((self.M, self.P))
        self.df_ratio = zeros((self.M, self.P))
        self.spec_flat = zeros((self.M, self.P))
        self.spec_ent = zeros((self.M, self.P))

        # function
        self.nfft = 2 ** (<int>(log(self.N) * self.invlog2))
        self.freq = linspace(0, 0.5 * fs, self.nfft)

        self.ihcut = <int>(floor(hi_cut / (fs / 2) * (self.nfft - 1)) + 1)  # high cutoff index
        self.ilcut = <int>(ceil(low_cut / (fs / 2) * (self.nfft - 1)))  # low cutoff index

        if self.ihcut > self.nfft:
            self.ihcut = <int>(self.nfft)
        self.lic2 = log(self.ihcut - self.ilcut) * self.invlog2

        self.sp_hat = fft.fft(x, 2 * self.nfft, axis=1)
        self.sp_norm = real(self.sp_hat[:, self.ilcut:self.ihcut, :] 
                            * conjugate(self.sp_hat[:, self.ilcut:self.ihcut, :]))
        
        self.sp_norm = self.sp_norm / nsum(self.sp_norm, axis=1, keepdims=True) + 1e-10

        self.imax = argmax(self.sp_norm, axis=1)

        # TODO uncomment, figure out how to implement
        # self.base_run = True

    cpdef get_dominant_freq(self, const double[:, :, :] x, double fs, double low_cut, double hi_cut):
        if not self.base_run:
            self._base_fn(x, fs, low_cut, hi_cut)
    
        for self.i in range(self.M):
            for self.k in range(self.P):
                self.maxf[self.i, self.k] = self.freq[self.imax[self.i, self.k] + self.ilcut]
        
        return self.maxf
    
    cpdef get_dominant_freq_value(self, const double[:, :, :] x, double fs, double low_cut, double hi_cut):
        if not self.base_run:
            self._base_fn(x, fs, low_cut, hi_cut)
        
        for self.i in range(self.M):
            for self.k in range(self.P):
                self.maxfv[self.i, self.k] = self.sp_norm[self.i, self.imax[self.i, self.k], self.k]
        
        return self.maxfv

    cpdef get_power(self, const double[:, :, :] x, double fs, double low_cut, double hi_cut):
        if not self.base_run:
            self._base_fn(x, fs, low_cut, hi_cut)

        # TODO remove the base run setting
        self.base_run = True
        _ = self.get_dominant_freq(x, fs, low_cut, hi_cut)
        self.base_run = False  # turn off for now
        
        for self.i in range(self.M):
            for self.j in range(self.ihcut - self.ilcut):
                for self.k in range(self.P):
                    if ((self.maxf[self.i, self.k] - 0.5) < self.freq[self.j + self.ilcut] < (self.maxf[self.i, self.k] + 0.5)):
                        self.df_ratio[self.i, self.k] += self.sp_norm[self.i, self.j, self.k]
    
        return self.df_ratio
    
    cpdef get_spectral_flatness(self, const double[:, :, :] x, double fs, double low_cut, double hi_cut):
        if not self.base_run:
            self._base_fn(x, fs, low_cut, hi_cut)
        
        for self.i in range(self.M):
            for self.k in range(self.P):
                self.mean = 0.
                mean_1d(self.sp_norm[self.i, :, self.k], &self.mean)
                self.spec_flat[self.i, self.k] = 10. * log(gmean(self.sp_norm[self.i, :, self.k]) / self.mean) / log(10.0)
        
        return self.spec_flat
    
    cpdef get_spectral_entropy(self, const double[:, :, :] x, double fs, double low_cut, double hi_cut):
        if not self.base_run:
            self._base_fn(x, fs, low_cut, hi_cut)
        
        for self.i in range(self.M):
            for self.j in range(self.ihcut - self.ilcut):
                for self.k in range(self.P):
                    self.logps = log(self.sp_norm[self.i, self.j, self.k]) * self.invlog2
                    self.spec_ent[self.i, self.k] -= self.logps * self.sp_norm[self.i, self.j, self.k]
            for self.k in range(self.P):
                self.spec_ent[self.i, self.k] /= self.lic2
        
        return self.spec_ent

In [4]:
from freq import dominantfrequency, dominantfrequencyvalue
from freq import powerspectralsum, spectralentropy, spectralflatness

In [5]:
from freq2 import dominantfrequency as df2, dominantfrequencyvalue as dfv2
from freq2 import powerspectralsum as pss2, spectralentropy as se2, spectralflatness as sf2

In [8]:
np.random.seed(67)
x = np.random.rand(50000, 150, 3)
xf = np.asfortranarray(x.transpose([1, 2, 0]))

nfft = 2 ** int(np.log2(x.shape[1]))

In [9]:
%timeit FrequencyFeatures().get_dominant_freq(x, 50.0, 1.0, 4.0)
%timeit dominantfrequency(xf, nfft, 50.0, 1.0, 4.0)
%timeit df2(xf, nfft, 50.0, 1.0, 4.0)

508 ms ± 879 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
309 ms ± 203 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
200 ms ± 163 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%timeit FrequencyFeatures().get_dominant_freq_value(x, 50.0, 1.0, 4.0)
%timeit dominantfrequencyvalue(xf, nfft, 50.0, 1.0, 4.0)
%timeit dfv2(xf, nfft, 50.0, 1.0, 4.0)

508 ms ± 116 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
308 ms ± 129 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
194 ms ± 30.3 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%timeit FrequencyFeatures().get_power(x, 50.0, 1.0, 4.0)
%timeit powerspectralsum(xf, nfft, 50.0, 1.0, 4.0)
%timeit pss2(xf, nfft, 50.0, 1.0, 4.0)

514 ms ± 296 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
313 ms ± 153 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
199 ms ± 63.9 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%timeit FrequencyFeatures().get_spectral_entropy(x, 50.0, 1.0, 4.0)
%timeit spectralentropy(xf, nfft, 50.0, 1.0, 4.0)
%timeit se2(xf, nfft, 50.0, 1.0, 4.0)

562 ms ± 360 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
352 ms ± 160 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
236 ms ± 42.9 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%timeit FrequencyFeatures().get_spectral_flatness(x, 50.0, 1.0, 4.0)
%timeit spectralflatness(xf, nfft, 50.0, 1.0, 4.0)
%timeit sf2(xf, nfft, 50.0, 1.0, 4.0)

597 ms ± 80.5 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
325 ms ± 167 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
210 ms ± 159 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
low = 1.0
high = 4.0
fs = 50.0

print(np.allclose(
    dominantfrequency(xf, nfft, fs, low, high).T,
    np.array(FrequencyFeatures().get_dominant_freq(x, fs, low, high))
))

print(np.allclose(
    dominantfrequencyvalue(xf, nfft, fs, low, high).T,
    np.array(FrequencyFeatures().get_dominant_freq_value(x, fs, low, high))
))

print(np.allclose(
    powerspectralsum(xf, nfft, fs, low, high).T,
    np.array(FrequencyFeatures().get_power(x, fs, low, high))
))

print(np.allclose(
    spectralentropy(xf, nfft, fs, low, high).T,
    np.array(FrequencyFeatures().get_spectral_entropy(x, fs, low, high))
))

print(np.allclose(
    spectralflatness(xf, nfft, fs, low, high).T,
    np.array(FrequencyFeatures().get_spectral_flatness(x, fs, low, high)),
    atol=1e-8,
    rtol=1e-3
))

True
True
True
True
True


In [15]:
low = 1.0
high = 4.0
fs = 50.0

print(np.allclose(
    df2(xf, nfft, fs, low, high).T,
    np.array(FrequencyFeatures().get_dominant_freq(x, fs, low, high))
))

print(np.allclose(
    dfv2(xf, nfft, fs, low, high).T,
    np.array(FrequencyFeatures().get_dominant_freq_value(x, fs, low, high))
))

print(np.allclose(
    pss2(xf, nfft, fs, low, high).T,
    np.array(FrequencyFeatures().get_power(x, fs, low, high))
))

print(np.allclose(
    se2(xf, nfft, fs, low, high).T,
    np.array(FrequencyFeatures().get_spectral_entropy(x, fs, low, high))
))

print(np.allclose(
    sf2(xf, nfft, fs, low, high).T,
    np.array(FrequencyFeatures().get_spectral_flatness(x, fs, low, high)),
    atol=1e-8,
    rtol=1e-3
))

True
True
True
True
True


In [6]:
%matplotlib widget

In [7]:
import matplotlib.pyplot as plt

In [8]:
from scipy.misc import electrocardiogram
from scipy.signal import sosfiltfilt, butter

In [19]:
sos = butter(4, 0.4 / 30, btype='high', output='sos')

ecg = sosfiltfilt(sos, electrocardiogram()[::6])
xecg = np.zeros((3, 180, 1))
xecgf = np.asfortranarray(xecg.transpose([1, 2, 0]))

for i in range(3):
    xecg[i, :, 0] = ecg[i * 180:i * 180 + 180]


nfft = 2 ** int(np.log2(xecg.shape[1]))
freq = np.linspace(0, 0.5 * 60, nfft, endpoint=False)
F = np.fft.fft(xecg, 2 * nfft, axis=1)
ps = np.real(F[:, :nfft] * np.conjugate(F[:, :nfft]))

In [34]:
lc = 1.13857
hc = 6.0

fs = 60.0

ihc = int((np.floor(hc / (fs / 2) * (nfft - 1)) + 1))
ilc = int((np.ceil(lc / (fs / 2) * (nfft - 1))))

ps /= np.sum(ps[:, ilc:ihc], axis=1, keepdims=True)

In [35]:
imx = np.argmax(ps[:, ilc:ihc], axis=1).ravel() + ilc
print(freq[imx])

[3.515625 5.859375 4.21875 ]


In [36]:
ps[:, ilc:ihc].max(axis=1).ravel()

array([0.18970462, 0.21305993, 0.3106129 ])

In [37]:
for i in range(3):
    s = 0
    for j in range(ilc, ihc):
        if (freq[imx[i]] - 0.5) < freq[j] < (freq[imx[i]] + 0.5):
            s += ps[i, j, 0]
    print(s, end='  ')

0.42277532854506256  0.34579681179352806  0.6641430484541447  

In [38]:
print(dominantfrequency(xecgf, nfft, 60.0, lc, hc).ravel())
print(np.array(FrequencyFeatures().get_dominant_freq(xecg, 60.0, lc, hc)).ravel())

[3.515625 5.859375 4.21875 ]
[3.515625 5.859375 4.21875 ]


In [39]:
print(dominantfrequencyvalue(xecgf, nfft, 60.0, lc, hc).ravel())
print(np.array(FrequencyFeatures().get_dominant_freq_value(xecg, 60.0, lc, hc)).ravel())

[0.18970463 0.21305992 0.31061291]
[0.18970462 0.21305993 0.3106129 ]


In [40]:
print(powerspectralsum(xecgf, nfft, 60.0, lc, hc).ravel())
print(np.array(FrequencyFeatures().get_power(xecg, 60.0, lc, hc)).ravel())

[0.42277532 0.34579681 0.66414304]
[0.42277533 0.34579681 0.66414305]


In [27]:
f, ax = plt.subplots(figsize=(10, 5))

for i in range(3):
    ax.plot(freq[:ilc+1], ps[i, :ilc+1], color=f'C{i}', alpha=0.3)
    ax.plot(freq[ilc:ihc+1], ps[i, ilc:ihc+1], color=f'C{i}')
    ax.plot(freq[ihc:], ps[i, ihc:], color=f'C{i}', alpha=0.3)
    
    imx = ps[i, ilc:ihc].argmax() + ilc
    ax.plot(freq[imx], ps[i, imx], 'o', color=f'C{i}')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
f, ax = plt.subplots(figsize=(10, 5))

for i in range(3):
    ax.plot(np.arange(0, 3, 1/60), xecg[i])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …